In [4]:
import sys
import os
import requests
import config

#!pip install timm==0.4.12

# replace with pip install library
from etab.utils.callbacks import *
from etab.baselines.models import *
from etab.datasets import *
import etab

import argparse
import warnings
warnings.filterwarnings("ignore")


if torch.cuda.is_available():
    
    print("GPU(s) available: ", torch.cuda.get_device_name())
    
else:
    
    print("No GPUs available")



cuda_device      = 0
device           = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")

import pickle

set_seeds(42)

GPU(s) available:  NVIDIA A100-SXM4-40GB


In [2]:
#echonet_dataset = echonet(view="A4CH")
#echonet_dataset.load_data()

# Pip install
# basic usage

# Add demo for data and functionalities:
# - show segmentations
# - show video gify

In [3]:
def train_baseline(model,
                   train_loader, 
                   valid_loader, 
                   task_code, 
                   callbacks,
                   n_epoch):
    
    save_base_dir = 'checkpoints'
    # Reload the pretrained network and freeze it except for its head.
    
    seg_tasks     = ["0", "1", "2"]
    class_tasks   = ["3", "4", "5", "6"]
    
    if task_code[-1] in seg_tasks:
        
        epoch_metric = ['f1', torchmetrics.JaccardIndex(num_classes=2)]
        optimizer    = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.001)
    
    else:
        
        epoch_metric = ['f1']
        optimizer    = optim.SGD(model.fc.parameters(), lr=learning_rate, weight_decay=0.001)

    # Saves everything into ./saves/cub200_resnet18_experiment
    save_path     = os.path.join(save_base_dir, task_code + "_" + backbone_type + "_target")

    
    loss_function = nn.CrossEntropyLoss()

    model_        = Model(model, 
                          optimizer, 
                          loss_function,
                          batch_metrics=['accuracy'], 
                          epoch_metrics=epoch_metric,
                          device=device)
    
    model_.fit_generator(train_loader, 
                         valid_loader, 
                         epochs=n_epoch, 
                         callbacks=callbacks)
    
    return model_

In [4]:
# benchmark task code

source_task      = "EA40"
target_task      = "CA45"
backbone_type    = "resnet50"

benchmark_task   = source_task + target_task

cuda_device      = 0
device           = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")

num_classes      = 200
batch_size       = 32
learning_rate    = 0.01
n_epoch          = 10

In [5]:
source_dataset, target_dataset        = prepare_benchmark_data(source_task=source_task, 
                                                               target_task=target_task)

In [6]:
source_train, source_val, source_test = ETAB_train_test_split(source_dataset, 
                                                              train_frac=0.6, 
                                                              val_frac=0.5)

target_train, target_val, target_test = ETAB_train_test_split(target_dataset, 
                                                              train_frac=0.6, 
                                                              val_frac=0.5)

In [7]:
source_train_loader  = DataLoader(source_train, batch_size=batch_size, num_workers=8, shuffle=True)
source_valid_loader  = DataLoader(source_val, batch_size=batch_size, num_workers=8)
source_test_loader   = DataLoader(source_test, batch_size=batch_size, num_workers=8)

target_train_loader  = DataLoader(target_train, batch_size=batch_size, num_workers=8, shuffle=True)
target_valid_loader  = DataLoader(target_val, batch_size=batch_size, num_workers=8)
target_test_loader   = DataLoader(target_test, batch_size=batch_size, num_workers=8)

In [8]:
# Source task

In [9]:
source_backbone  = prepare_ETAB_model(backbone_type, 
                                      pretrained=True, 
                                      mode="segmentation",
                                      num_classes=2)

In [10]:
callbacks        = init_callbacks(benchmark_task)

In [11]:
source_model     = train_baseline(source_backbone,
                                  source_train_loader, 
                                  source_valid_loader, 
                                  task_code=source_task, 
                                  callbacks=callbacks,
                                  n_epoch=n_epoch)

Epoch:  1/10 Train steps: 140 Val steps: 47 20.95s loss: 0.236740 acc: 91.088460 fscore_macro: 0.749394 jaccard_index: 0.641705 val_loss: 0.136479 val_acc: 95.028235 val_fscore_macro: 0.872467 val_jaccard_index: 0.787739
Epoch 1: val_loss improved from inf to 0.13648, saving file to checkpoints/EA40CA45/best_weight.ckpt
Epoch:  2/10 Train steps: 140 Val steps: 47 19.41s loss: 0.110880 acc: 95.918420 fscore_macro: 0.895260 jaccard_index: 0.820368 val_loss: 0.104998 val_acc: 96.069074 val_fscore_macro: 0.897062 val_jaccard_index: 0.823155
Epoch 2: val_loss improved from 0.13648 to 0.10500, saving file to checkpoints/EA40CA45/best_weight.ckpt
Epoch:  3/10 Train steps: 140 Val steps: 47 19.84s loss: 0.090333 acc: 96.536895 fscore_macro: 0.911439 jaccard_index: 0.844723 val_loss: 0.094365 val_acc: 96.323102 val_fscore_macro: 0.906630 val_jaccard_index: 0.837329
Epoch 3: val_loss improved from 0.10500 to 0.09437, saving file to checkpoints/EA40CA45/best_weight.ckpt
Epoch:  4/10 Train steps: 

In [12]:
target_model     = attach_head(source_backbone, 
                               backbone_type=backbone_type, 
                               source_mode="segmentation",
                               target_mode="classification", 
                               num_classes=2)

In [13]:
freeze_weights(target_model)

In [14]:
callbacks        = init_callbacks(benchmark_task)

In [15]:
target_model     = train_baseline(target_model, 
                                  target_train_loader,
                                  target_valid_loader,
                                  task_code=target_task, 
                                  callbacks=callbacks,
                                  n_epoch=n_epoch)

Epoch:  1/10 Train steps: 9 Val steps: 3 1.81s loss: 0.751541 acc: 51.851852 fscore_macro: 0.504125 val_loss: 0.782565 val_acc: 36.666667 val_fscore_macro: 0.268293
Epoch 1: val_loss improved from inf to 0.78256, saving file to checkpoints/EA40CA45/best_weight.ckpt
Epoch:  2/10 Train steps: 9 Val steps: 3 1.79s loss: 0.805765 acc: 49.259259 fscore_macro: 0.486671 val_loss: 0.742051 val_acc: 63.333333 val_fscore_macro: 0.387755
Epoch 2: val_loss improved from 0.78256 to 0.74205, saving file to checkpoints/EA40CA45/best_weight.ckpt
Epoch:  3/10 Train steps: 9 Val steps: 3 1.80s loss: 0.747555 acc: 56.666667 fscore_macro: 0.534476 val_loss: 0.647095 val_acc: 63.333334 val_fscore_macro: 0.414778
Epoch 3: val_loss improved from 0.74205 to 0.64709, saving file to checkpoints/EA40CA45/best_weight.ckpt
Epoch:  4/10 Train steps: 9 Val steps: 3 1.79s loss: 0.791618 acc: 54.444444 fscore_macro: 0.519941 val_loss: 0.958941 val_acc: 36.666667 val_fscore_macro: 0.268293
Epoch:  5/10 Train steps: 9 V

In [16]:
# make a single training function

In [17]:
#!zip -r etab.zip etab

In [18]:
from PIL import Image

def create_echo_clip(one_patient_echo, op_file_name):
    
    echo_frames = []

    for u in range(len(one_patient_echo)):
        
        echo_image  = one_patient_echo[u]

        echo_frames.append(PIL_transform(echo_frames.detach().numpy().astype(np.uint8)))
    
    echo_frames[0].save(op_file_name, 
                        save_all=True, 
                        append_images=echo_frames[1:], optimize=False, duration=50, loop=0)  
  

In [5]:
import os
import collections
import pandas
import torch
import numpy as np
import skimage.draw
import torchvision
import etab.utils.echonet
from skimage.color import rgb2gray, gray2rgb
import torchvision.transforms as T
from PIL import Image
import matplotlib.image as mpimg

from os import listdir
from os.path import isfile, join
import numpy as np
from os import listdir
from os.path import isfile, join
import cv2
import xml.etree.ElementTree as ET

In [6]:
def load_segmented_data(data_dir, n_train=None, concatenate=True, rgb=False, IMG_SIZE=224, n_frames=1):
    
    transform    = T.ToPILImage()
    pil_2_tensor = T.ToTensor()
    
    dataset      = Echo(root=data_dir + "/data/", target_type="SmallTrace")

    n_train      = len(dataset) if n_train is None else n_train
    
    videos       = []
    segments     = []

    for _ in range(n_train):
  
        current_video, current_trace = dataset.__getitem__(_) 

        if rgb:
            
            current_video, current_trace = torch.einsum('cnhw->nhwc', torch.tensor(current_video)/255)[:n_frames, :, :, :], torch.tensor(gray2rgb(current_trace))[:n_frames, :, :, :]
        
        else:
            
            current_video = torch.einsum('cnhw->nhwc', torch.tensor(current_video)/255)[:n_frames, :, :, :]                
            current_trace = torch.tensor(current_trace) 
            
            if current_video.shape[0]==1:
                
                current_video = current_video.squeeze(0)
                current_video = pil_2_tensor(transform(torch.einsum('hwc->chw', current_video)).resize((IMG_SIZE, IMG_SIZE)))
                current_trace = pil_2_tensor(transform(current_trace).resize((IMG_SIZE, IMG_SIZE))).squeeze(0)
            
            else:
                
                # loop over frames
                current_video = [pil_2_tensor(transform(torch.einsum('hwc->chw', current_video[kk, :, :, :])).resize((IMG_SIZE, IMG_SIZE))).unsqueeze(0) for kk in range(current_video.shape[0])]
                current_video = torch.cat(current_video, dim=0)
                current_trace = pil_2_tensor(transform(current_trace).resize((IMG_SIZE, IMG_SIZE))).squeeze(0)     
            

        videos.append(current_video.unsqueeze(0))
        segments.append(current_trace.unsqueeze(0))  
    
    if concatenate:
        
        videos       = torch.cat(videos)
        segments     = torch.cat(segments) 
        all_data_set = torch.cat([videos, segments], dim=1)
        
    else:
        
        all_data_set = [(videos[k].squeeze(0), segments[k].squeeze(0).type(torch.LongTensor)) for k in range(len(videos))]
    
    return all_data_set


In [7]:
data_ = load_segmented_data(data_dir=config.echonet_dir, 
                                       n_train=None, 
                                       concatenate=False, 
                                       rgb=False, 
                                       IMG_SIZE=224, n_frames=20)

In [ ]:
#from matplotlib import pyplot as plt

#plt.imshow(data_[0][0][0, :, :])

In [ ]:
#reate_echo_clip(one_patient_echo, op_file_name)

In [9]:
data_[0][0].shape

torch.Size([16, 3, 224, 224])